In [1]:
import tensorflow as tf
# Fixes bad convolution
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
inp = tf.constant([1, 3, 2])

#@tf.function(experimental_compile=True)
def solve(inp, until_turn=2020):
    ta = tf.TensorArray(tf.int32, size=until_turn)
    n = inp.shape[0]
    
    for i in tf.range(n):
        turn = i+1
        ta = ta.write(inp[i], turn)
    
    prev_x = 0

    for i in tf.range(n, until_turn-1):
        last_turn = i+1
        prev_turn = ta.read(prev_x)
        if prev_turn == 0:
            new_x = 0
        else:
            new_x = last_turn-prev_turn
        
        ta = ta.write(prev_x, last_turn)
        prev_x = new_x
    ta = ta.close()
    return prev_x

assert solve(tf.constant([0, 3, 6])) == 436
assert solve(tf.constant([1, 3, 2])) == 1
assert solve(tf.constant([2, 1, 3])) == 10
assert solve(tf.constant([1, 2, 3])) == 27

In [5]:
%timeit solve(tf.constant([2,0,1,9,5,19]))
solve(tf.constant([2,0,1,9,5,19]))

512 ms ± 1.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


<tf.Tensor: shape=(), dtype=int32, numpy=1009>

In [ ]:
solve(tf.constant([2,0,1,9,5,19]), )